# 爬蟲1：爬PTT

In [1]:
import requests

In [2]:
# PTT home_sale的目錄頁面
URL = "https://www.ptt.cc/bbs/home-sale/index{}.html"

In [3]:
# 3788是指查詢當時PTT的最後一頁的數字
resp = requests.get(URL.format(3788))

In [4]:
# 列出index3788.html的表頭
resp.headers

{'Date': 'Tue, 09 Jan 2018 07:46:01 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Set-Cookie': '__cfduid=dd51d1b68d0cafa04e780ff0745d264721515483961; expires=Wed, 09-Jan-19 07:46:01 GMT; path=/; domain=.ptt.cc; HttpOnly', 'Alternate-Protocol': '443:npn-spdy/3', 'Strict-Transport-Security': 'max-age=0', 'X-Content-Type-Options': 'nosniff', 'Server': 'cloudflare', 'CF-RAY': '3da5c7c8ab865414-LAX', 'Content-Encoding': 'gzip'}

In [5]:
# 列出該頁的content， 若沒有用utf-8的話會顯示成ascii code
resp.content.decode('utf-8')[:200]

'<!DOCTYPE html>\n<html>\n\t<head>\n\t\t<meta charset="utf-8">\n\t\t\n\n<meta name="viewport" content="width=device-width, initial-scale=1">\n\n<title>看板 home-sale 文章列表 - 批踢踢實業坊</title>\n\n<link rel="stylesheet" type'

In [6]:
!mkdir -p /tmp/ptt_data/home_sale/

# 根據對象網站HTTP 的方法決定抓取方式，像是 get, post ...

requests.get(): get(url,**kwargs)。 全頁面抓取則不設定參數，若只抓header則參數寫：headers=headers、 有許多參數可設定

# with open（檔案存放or讀寫位置,'w' or 'r'） as f: 的使用方式

In [7]:
LAST_PAGE = 3788
PAGE = 5
DATA_DIR = '/tmp/ptt_data/home_sale/'

# page_no最多可抓0 ~ 3782筆，利用for迴圈來存放預存的筆數
for page_no in range(LAST_PAGE, LAST_PAGE - PAGE, -1):
    print(page_no)

# URL = "https://www.ptt.cc/bbs/home-sale/index{}.html"
# 因為有多筆網頁， 所以創立resp變數來存放多筆網頁url
    resp = requests.get(URL.format(page_no))
    
# 把爬下來的HTML文字檔儲存到資料夾，先指定檔案存放位置及檔案名稱
    with open(DATA_DIR + 'index{}.html'.format(page_no), 'w') as f:
# 將resp的內文（.text）寫入
        f.write(resp.text)

3788
3787
3786
3785
3784


# 要注意此時的resp.text只存留最後一筆（3778）的內文資料！！！

In [8]:
!ls /tmp/ptt_data/home_sale

bunch	   index3784.html  index3786.html  index3788.html
index3784  index3785.html  index3787.html


# 目錄頁面抓完後，發現內文頁面的頁碼規則不是連號 -> import re 來使用Regular Expression

In [9]:
 # 'http://ptt.cc/bbs/LCD/M.1514102861.A.55A.html',
 # 'http://ptt.cc/bbs/LCD/M.1514175646.A.53B.html',
 # 'http://ptt.cc/bbs/LCD/M.1514183175.A.B2F.html',
 # 'http://ptt.cc/bbs/LCD/M.1514183767.A.506.html',
 # 'http://ptt.cc/bbs/LCD/M.1514203342.A.0FB.html',

In [10]:
import re

#  re.findall(pattern, string, flags=0)

Return all non-overlapping matches of pattern in string, as a list of strings. The string is scanned left-to-right, and matches are returned in the order found. If one or more groups are present in the pattern, return a list of groups; this will be a list of tuples if the pattern has more than one group.

Note: Due to the limitation of the current implementation the character following an empty match is not included in a next match, so findall(r'^|\w+', 'two words') returns ['', 'wo', 'words'] (note missed “t”). This is changed in Python 3.7. 

In [11]:
# 該函數可以以列表的形式返回所有匹配模式的項：

pat = '[a-zA-Z]+'
text = 'Hello, hm...this is Tom speaking, who are you?'
re.findall(pat, text)
# result ['Hello', 'hm', 'this', 'is', 'Tom', 'speaking', 'who', 'are', 'you']


['Hello', 'hm', 'this', 'is', 'Tom', 'speaking', 'who', 'are', 'you']

In [12]:
URL2 = 'https://www.ptt.cc/bbs/home-sale/M.1515{}.html'

# 先來練習單頁爬蟲
寫一個function， 可以將標題列的url內文（resp.txt）利用re的findall找出符合該規則的所有網址找出來， 再把所有的內文網址組合成符合網址url的格式後除存到detail_page_links

In [13]:
import re

import requests

def get_list_page(url):

    HOST = "https://www.ptt.cc"

    resp = requests.get(url)
    links = re.findall('<a href="(/bbs/home-sale/M.+\.html)">.+</a>', resp.text)
    detail_page_links = [HOST + link for link in links]
    return detail_page_links

In [14]:
# 抓取index3784.html內所有的內文網址url
get_list_page("https://www.ptt.cc/bbs/home-sale/index3784.html")

['https://www.ptt.cc/bbs/home-sale/M.1515216249.A.874.html',
 'https://www.ptt.cc/bbs/home-sale/M.1515218808.A.29B.html',
 'https://www.ptt.cc/bbs/home-sale/M.1515220517.A.4DB.html',
 'https://www.ptt.cc/bbs/home-sale/M.1515222349.A.303.html',
 'https://www.ptt.cc/bbs/home-sale/M.1515222778.A.3CE.html',
 'https://www.ptt.cc/bbs/home-sale/M.1515223481.A.E9F.html',
 'https://www.ptt.cc/bbs/home-sale/M.1515224309.A.221.html',
 'https://www.ptt.cc/bbs/home-sale/M.1515224893.A.FC3.html',
 'https://www.ptt.cc/bbs/home-sale/M.1515225219.A.1A2.html',
 'https://www.ptt.cc/bbs/home-sale/M.1515226362.A.DA5.html',
 'https://www.ptt.cc/bbs/home-sale/M.1515232091.A.075.html',
 'https://www.ptt.cc/bbs/home-sale/M.1515232447.A.CA3.html',
 'https://www.ptt.cc/bbs/home-sale/M.1515234326.A.657.html',
 'https://www.ptt.cc/bbs/home-sale/M.1515237400.A.9B9.html',
 'https://www.ptt.cc/bbs/home-sale/M.1515238322.A.851.html',
 'https://www.ptt.cc/bbs/home-sale/M.1515238360.A.261.html',
 'https://www.ptt.cc/bbs

In [15]:
# 將以上的所有網址寫入3784的資料夾內
!mkdir -p /tmp/ptt_data/home_sale/index3784

In [16]:
list3784 = get_list_page("https://www.ptt.cc/bbs/home-sale/index3784.html")

In [17]:
# 將資料寫入硬碟內

DATA_PATH = '/tmp/ptt_data/home_sale/index3784'

def dump_page(url):

    filename = "_".join(url.split('/')[-1].split('.')[:-1]) + '.html'
    resp = requests.get(url)

    with open(DATA_PATH + '/' + filename, 'w') as f:
        f.write(resp.text)
    return filename

if __name__ == "__main__":
    for link in list3784:
        print(link)
        dump_page(link)

https://www.ptt.cc/bbs/home-sale/M.1515216249.A.874.html
https://www.ptt.cc/bbs/home-sale/M.1515218808.A.29B.html
https://www.ptt.cc/bbs/home-sale/M.1515220517.A.4DB.html
https://www.ptt.cc/bbs/home-sale/M.1515222349.A.303.html
https://www.ptt.cc/bbs/home-sale/M.1515222778.A.3CE.html
https://www.ptt.cc/bbs/home-sale/M.1515223481.A.E9F.html
https://www.ptt.cc/bbs/home-sale/M.1515224309.A.221.html
https://www.ptt.cc/bbs/home-sale/M.1515224893.A.FC3.html
https://www.ptt.cc/bbs/home-sale/M.1515225219.A.1A2.html
https://www.ptt.cc/bbs/home-sale/M.1515226362.A.DA5.html
https://www.ptt.cc/bbs/home-sale/M.1515232091.A.075.html
https://www.ptt.cc/bbs/home-sale/M.1515232447.A.CA3.html
https://www.ptt.cc/bbs/home-sale/M.1515234326.A.657.html
https://www.ptt.cc/bbs/home-sale/M.1515237400.A.9B9.html
https://www.ptt.cc/bbs/home-sale/M.1515238322.A.851.html
https://www.ptt.cc/bbs/home-sale/M.1515238360.A.261.html
https://www.ptt.cc/bbs/home-sale/M.1515238373.A.E0F.html
https://www.ptt.cc/bbs/home-sal

# 爬PTT一整個區間的文章
這時候get_list_page（）已經不是單一目錄頁了，所以需要改成變動性的寫法

In [21]:
# 建立新資料夾
!mkdir -p /tmp/ptt_data/home_sale/bunch

In [9]:
import re
import requests

# 設定存檔位置
DATA_PATH = '/tmp/ptt_data/home_sale/bunch'
# 設定抓取目錄頁的區間數
CRAWL_PAGE_CNT = 5
# 目錄頁的url格式
URL_TEMPLATE = "https://www.ptt.cc/bbs/home-sale/index{}.html"
HOST = "https://www.ptt.cc"

In [10]:
"""
1. 剛進home_sale板的時候為index.html, 無法得知實際最新頁面數， 可由"上頁"按鈕去inspect出"上頁"的實際url，再+1即為最新頁面
2. href="/bbs/home-sale/index3789.html">‹ 上頁 (note:不同瀏覽器看到的結果不一樣， 所以若抓不到資料要換板測試)
3. 『‹』的符號在python無法被搜尋到,故須改成 『&lsaquo;』 （ ‹ = &lsaquo; ）
"""
# 取得現在PTT home_sale板塊的總頁數
def get_total_page_cnt():
    url = URL_TEMPLATE.format('')
    resp = requests.get(url)

    total_page_cnt = int(re.findall('href="/bbs/home-sale/index(\d+).html">&lsaquo; 上頁', resp.text)[0]) + 1
    return total_page_cnt

In [14]:
# GET function，將目錄頁url丟進去後可取得內文頁的連結們
def get_list_page(url):
    resp = requests.get(url)
    links = re.findall('<a href="(/bbs/home-sale/M.+\.html)">.+</a>', resp.text)
    detail_page_links = [HOST + link for link in links]
    return detail_page_links

In [15]:
def dump_page(url):

    filename = "_".join(url.split('/')[-1].split('.')[:-1]) + '.html'
    resp = requests.get(url)

    with open(DATA_PATH + '/' + filename, 'w') as f:
        f.write(resp.text)
    return filename

if __name__ == "__main__":
    total_page_cnt = get_total_page_cnt()

    for pg in range(total_page_cnt, total_page_cnt - CRAWL_PAGE_CNT, -1):
        url = URL_TEMPLATE.format(pg)
        for link in get_list_page(url):
            print(link)
            dump_page(link)

https://www.ptt.cc/bbs/home-sale/M.1515465418.A.666.html
https://www.ptt.cc/bbs/home-sale/M.1515466560.A.60A.html
https://www.ptt.cc/bbs/home-sale/M.1515468997.A.8B0.html
https://www.ptt.cc/bbs/home-sale/M.1515469034.A.F8C.html
https://www.ptt.cc/bbs/home-sale/M.1515469880.A.14B.html
https://www.ptt.cc/bbs/home-sale/M.1515469948.A.0DA.html
https://www.ptt.cc/bbs/home-sale/M.1515469986.A.ADD.html
https://www.ptt.cc/bbs/home-sale/M.1515470741.A.0BF.html
https://www.ptt.cc/bbs/home-sale/M.1515476772.A.318.html
https://www.ptt.cc/bbs/home-sale/M.1515478603.A.2E4.html
https://www.ptt.cc/bbs/home-sale/M.1515483103.A.C8F.html
https://www.ptt.cc/bbs/home-sale/M.1515485471.A.6EA.html
https://www.ptt.cc/bbs/home-sale/M.1515487938.A.0E9.html
https://www.ptt.cc/bbs/home-sale/M.1493615587.A.2B3.html
https://www.ptt.cc/bbs/home-sale/M.1502285316.A.D93.html
https://www.ptt.cc/bbs/home-sale/M.1507295887.A.D3D.html
https://www.ptt.cc/bbs/home-sale/M.1514756397.A.FAF.html
https://www.ptt.cc/bbs/home-sal

# 8掛板要求18歲以上，需要被PTT確認後才能進入
會先在index.html頁面被要求按下"滿18歲"按鈕。 進入inspect模式觀察network選項，after press "submit" buttom, notice the action that we send that to PTT. In Over 18 page, we can see "set-cookie:over18=1; Path=/" in Response Headers, so we need to send request header with the info of "set-cookie:over18=1; Path=/"

In [16]:
import re

import requests

DATA_PATH = '/tmp/ptt_data/Gossip'
CRAWL_PAGE_CNT = 3
URL_TEMPLATE = "https://www.ptt.cc/bbs/Gossiping/index{}.html"
HOST = "https://www.ptt.cc"

In [17]:
# 建立存放位置
!mkdir -p /tmp/ptt_data/Gossip

In [19]:
# 設定手動header contents

def get_w_cookie(url):
    """GET HTTP url with custom header containing cookie info for PTT Parameters
    url : str
        PTT八卦版需要驗證年齡的URL，
        如：https://www.ptt.cc/bbs/Gossiping/index31063.html
    """
    custom_headers = {
        "cookie": "over18=1;"
    }
    resp = requests.get(url, headers=custom_headers)
    return resp

In [20]:
# 取得現在ptt板塊的總頁數, 由於第一次直接進入時都會被問要按滿18歲,所以 requests.get(url, headers=custom_headers)

def get_total_page_cnt():
    url = URL_TEMPLATE.format('')
    resp = get_w_cookie(url)

    total_page_cnt = int(re.findall('href="/bbs/Gossiping/index(\d+).html">&lsaquo; 上頁', resp.text)[0]) + 1
    return total_page_cnt

In [22]:
# GET列表頁，取得內文頁的連結們, 由於第一次直接進入時都會被問要按滿18歲,所以 requests.get(url, headers=custom_headers)

def get_list_page(url):
    resp = get_w_cookie(url)
    links = re.findall('<a href="(/bbs/Gossiping/M.+\.html)">.+</a>', resp.text)
    detail_page_links = [HOST + link for link in links]
    return detail_page_links

In [23]:
# 寫入資料

def dump_page(url):
    filename = "_".join(url.split('/')[-1].split('.')[:-1]) + '.html'
    resp = get_w_cookie(url)

    with open(DATA_PATH + '/' + filename, 'w') as f:
        f.write(resp.text)
    return filename

if __name__ == "__main__":
    total_page_cnt = get_total_page_cnt()

    for pg in range(total_page_cnt, total_page_cnt - CRAWL_PAGE_CNT, -1):
        url = URL_TEMPLATE.format(pg)
        for link in get_list_page(url):
            print(link)
            dump_page(link)

https://www.ptt.cc/bbs/Gossiping/M.1515492930.A.847.html
https://www.ptt.cc/bbs/Gossiping/M.1515492962.A.CAD.html
https://www.ptt.cc/bbs/Gossiping/M.1510415718.A.D77.html
https://www.ptt.cc/bbs/Gossiping/M.1515166066.A.4EF.html
https://www.ptt.cc/bbs/Gossiping/M.1514738804.A.4A0.html
https://www.ptt.cc/bbs/Gossiping/M.1515492559.A.9FE.html
https://www.ptt.cc/bbs/Gossiping/M.1515492566.A.0CF.html
https://www.ptt.cc/bbs/Gossiping/M.1515492571.A.846.html
https://www.ptt.cc/bbs/Gossiping/M.1515492597.A.CDA.html
https://www.ptt.cc/bbs/Gossiping/M.1515492605.A.8C9.html
https://www.ptt.cc/bbs/Gossiping/M.1515492630.A.60A.html
https://www.ptt.cc/bbs/Gossiping/M.1515492654.A.5B8.html
https://www.ptt.cc/bbs/Gossiping/M.1515492667.A.A16.html
https://www.ptt.cc/bbs/Gossiping/M.1515492670.A.A79.html
https://www.ptt.cc/bbs/Gossiping/M.1515492743.A.861.html
https://www.ptt.cc/bbs/Gossiping/M.1515492763.A.610.html
https://www.ptt.cc/bbs/Gossiping/M.1515492763.A.380.html
https://www.ptt.cc/bbs/Gossipin